梯度消失与梯度爆炸
===

# 1.梯度更新规则
梯度消失的根源在于深度神经网络和反向传播。我们知道在处理复杂任务上，深度网络比浅层的网络具有更好的效果。但是，目前优化神经网络的方法都是基于反向传播的思想，即根据损失函数计算的误差通过梯度反向传播的方式，指导深度网络权值的更新优化。

首先，深层网络由许多非线性层堆叠而来，每一层非线性层都可以视为是一个非线性函数$f(x)$(非线性来自于非线性激活函数），因此整个深度网络可以视为是一个复合的非线性多元函数
$$
F(x)=f_n(...f_3(f_2(f_1(x) \times \theta_1 + b) \times \theta_2 + b)...)
$$
我们最终的目的是希望这个多元函数可以很好的完成输入到输出之间的映射。

假设不同的输入，输出的最优解是$g(x)$，那么，优化深度网络就是为了寻找到合适的权值，满足$Loss=L(g(x),F(x))$,$F(x)$取得极小值点，比如最简单的损失函数
$$Loss=||g(x)-f(x)||_2^2$$
对于这种数学寻找最小值问题，采用梯度下降的方法再适合不过了

# 2.梯度消失与爆炸
在两种情况下，梯度消失经常出现：深层网络与不适合的损失函数，比如Sigmoid。梯度爆炸一般在两种情况下出现:深层网络和权值初始值太大。

## 2.1.深层网络
![image](Images/03/00/06_001.png)
图中是一个四层的全连接网络，假设每一层网络激活后的输出为$f_i(x)$,其中i为第i层，x代表第i层的输入，也就是第$i-1$层的输出，f是激活函数，那么就可以得出
$$f_{i+1}=f(f_i \times \omega_{i+1} + b_{i+1})$$
简单记作
$$f_{i+1}=f(f_i \times \omega_{i+1})$$
BP算法基于梯度下降策略，以目标的负梯度方向对参数进行调整，参数的更新为$\omega \leftarrow \omega + \Delta{\omega}$，给定学习率为$\alpha$，得出
$$\Delta{\omega}=-\alpha \times \frac{\partial{Loss}}{\partial{\omega}}$$
如果要更新第二个隐藏层的权值信息根据链式求导法则，更新梯度信息
$$\Delta{\omega_2}=\frac{\partial{Loss}}{\partial{\omega_2}}=\frac{\partial{Loss}}{\partial{f_4}} \times \frac{\partial{f_4}}{\partial{f_3}} \times \frac{\partial{f_3}}{\partial{f_2}} \times \frac{\partial{f_2}}{\partial{\omega_2}}$$
很容易看出来$\frac{\partial{f_2}}{\partial{\omega_2}}=\frac{\partial{f}}{\partial{(f_1 \times \omega_2)}} \times f_1$，即第二层的输入
所以说，$\frac{\partial{f_4}}{\partial{f_3}}$就是对已获函数进行求导，如果此部分大于1，那么层数增多的时候，最终求出的梯度更新将以指数形式增加，发生了梯度爆炸，如果此部分小于1，随着层数增多，求出的梯度更新将以指数衰减，发生梯度消失。

总结：从深层网络角度来讲，不同的层学习的速度差异很大，表现为网络中靠近输出的层学习的情况很好，靠近输入的层学习的很慢，有时甚至训练了很久，前几层的权值和刚开始随机初始化的值差不多。因此，梯度消失、爆炸，其根本原因在于反向传播训练法则，属于先天不足

Hinton提出capsule的原因就是为了彻底抛弃反向传播

## 2.2.激活函数
其实也注意到了，上文中提到计算权值更新信息的时候需要计算前层偏导信息，因此如果激活函数选择不合适，比如使用sigmoid，梯度消失就会很明显了

原因看下图，左图是sigmoid的损失函数图，右边是其导数的图像，如果使用sigmoid作为损失函数，其梯度是不可能超过0.25的，这样经过链式求导之后，很容易发生梯度消失
![image](Images/03/00/06_002.jpeg)
Sigmoid的数学表达式$\frac{1}{1+e^{-x}}$

同理，tanh作为激活函数，它的导数图如下，可以看出，tanh比sigmoid要好一些，但是它的导数仍然是小于1的。tanh数学表达为$\frac{e^x-e^{-x}}{e^x+e^{-x}}$
![image](Images/03/00/06_003.png)

# 3.梯度消失与梯度爆炸的解决方案

## 3.1.预训练加微调
此方法来自Hinton在2006年发表的一篇论文，Hinton为了解决梯度的问题，提出采取无监督逐层训练方法，其基本思想是每次训练一层隐节点，训练时将上一层隐节点的输出作为输入，而本层隐节点的输出作为下一层隐节点的输入，此过程就是逐层“预训练”（pre-training）；在预训练完成后，再对整个网络进行“微调”（fine-tunning）。Hinton在训练深度信念网络（Deep Belief Networks中，使用了这个方法，在各层预训练完成后，再利用BP算法对整个网络进行训练。此思想相当于是先寻找局部最优，然后整合起来寻找全局最优，此方法有一定的好处，但是目前应用的不是很多了。

## 3.2.梯度剪切、正则
梯度剪切这个方案主要是针对梯度爆炸提出的，其思想是设置一个梯度剪切阈值，然后更新梯度的时候，如果梯度超过这个阈值，那么就将其强制限制在这个范围之内。这可以防止梯度爆炸。

另外一种解决梯度爆炸的手段是采用权重正则化（weithts regularization）比较常见的是l1 l1l1正则，和l2 l2l2正则，在各个深度框架中都有相应的API可以使用正则化，比如在tensorflow tensorflowtensorflow中，若搭建网络的时候已经设置了正则化参数，则调用以下代码可以直接计算出正则损失：

正则化是通过对网络权重做正则限制过拟合，仔细看正则项在损失函数的形式
$$Loss=(y-W^Tx)^2+\alpha||W||^2$$
如果发生梯度爆炸，权值的范数就会变的非常大，通过正则化项，可以部分限制梯度爆炸的发生。事实上，在深度神经网络中，往往是梯度消失出现的更多一些。

## 3.3.使用relu、leakyrelu、elu等激活函数
## 3.4.使用BatchNorm
## 3.5.残差结构